## Logistic Regression Example

以 MNIST 为例，将 (28, 28) 转置成 (784, 1) 个 fetures 来处理。

In [17]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [18]:
train_data, test_data = tf.keras.datasets.mnist.load_data()

In [74]:
g1 = tf.Graph()
with g1.as_default():
    dataset = tf.data.Dataset.from_tensor_slices(train_data)
    iterator = tf.data.make_one_shot_iterator(dataset)
    next_element = iterator.get_next()
    
    W = tf.Variable(tf.random.normal([10, 784]), name="weights")
    
    with tf.Session() as sess:
        init_op = tf.global_variables_initializer()
        sess.run(init_op)
    

### 如何使用一个 Initializer

In [38]:
# 实例化一个 initializer, 但是想成为计算图中的一个节点，还需要我们调用相应的 API 函数。
# 如调用 __call__()

g2 = tf.Graph()
with g2.as_default():
    weights = tf.random_normal_initializer()
    init = weights((2, 3))
    
    with tf.Session() as sess:
        print(sess.run(init))

[[-1.0230905   0.52107584  0.38565353]
 [ 0.11472052  1.0831474   1.5300333 ]]


### 如何将 Initializer 与 tf.Variable 相结合

其实查看文档可以知道每一个 `tf.Variable` 对象都且个 `initializer` OP 可以让我们进行初始化。

In [71]:
g3 = tf.Graph()
with g3.as_default():
    w = tf.Variable(tf.random.normal((3,3), name="weights"))
    eye3 = tf.eye(3)
    a = tf.matmul(w, eye3)
    init_op = w.initializer
    
    with tf.Session() as sess:
        sess.run(init_op)   # 一定要记得初始化
        print(sess.run(a))

[[ 0.24014388 -0.0208146  -0.34707728]
 [-0.98216105  0.5356388  -0.15996137]
 [-0.736439   -0.19126625  0.0421228 ]]


有时候，我不必为所有 Variables 都单独执行 op。 我可以使用一个全局的 op 来完成初始化。

In [70]:
g4 = tf.Graph()
with g4.as_default():
    w = tf.Variable(tf.random.normal((3,3), name="weights"))
    e = tf.Variable(tf.eye(3), name="indentity")
    a = tf.matmul(w, e)
    
    init_all_op = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init_all_op)   # 一定要记得初始化
        print(sess.run(a))

[[-1.4287577e+00 -6.6445392e-01 -9.6790987e-01]
 [-9.7193003e-01 -5.9794134e-01 -1.0857213e+00]
 [-1.7487629e-01  1.3640827e-03 -1.2540774e-02]]


In [37]:
# Parameters
learning_rate = 0.01
epochs = 25
batch_size = 100

x = tf.placeholder(tf.float32, [784, 1])
y = tf.placeholder(tf.float32, [10, 1])

# Set model weights
W = tf.Variable(tf.zeros([10, 784]))
b = tf.Variable(tf.zeros([10]))

# Construct model
pred = tf.nn.softmax(tf.matmul(W, x) + b)

# Minimize error using cross entropy
cost = tf.reduce_sum(-tf.reduce_sum(y * tf.log(pred)))

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()